In [6]:
import os
import pandas as pd

from dotenv import load_dotenv
from routingGA import RoutingGA
from GALib import GALib
from data import addresses
from routingGA import RoutingGA
from distancesAPI import MapsAPI

%reload_ext autoreload
%autoreload 2

load_dotenv()

True

In [7]:
refetch_distances = False

In [8]:
mapsAPI = MapsAPI(os.getenv('API_KEY'))
address_chunks = MapsAPI.split_in_chunks(addresses, 25)
distances = mapsAPI.split_distance_request(address_chunks) if refetch_distances else MapsAPI.get_from_cache()
distances = MapsAPI.convert_to_symmetric(distances)

In [9]:
df = pd.DataFrame(distances)

In [10]:
routingGA = RoutingGA(popSize=500, qtyLocations=len(distances) - 1, qtyRoutes=5,
                          maxGenerations=2000, selectionK=3, mutationRate=0.02, optRate=0.02,distances=distances)
                           
lib = GALib(routingGA=routingGA,
            libPath=os.getenv('LIB_PATH'))
            
result = lib.run()

In [11]:
routingGA.calculate_total_distance()

35903